# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
read_path = os.path.join("output_data/cities.csv")
main_frame = pd.read_csv(read_path, low_memory=False)
main_frame.head()

,City_ID,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed
0,0,MU,1584087856,-20.52,57.52,75,94,297.15,1.50
1,1,MV,1584087856,-0.60,73.08,0,68,302.06,4.64
2,2,TO,1584087856,-21.20,-175.20,75,94,299.15,4.10
3,3,US,1584087844,32.70,-103.14,75,100,286.48,2.10
4,4,PE,1584087857,-9.09,-78.58,100,69,296.94,1.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
type(main_frame['Humidity'][1])

numpy.int64

In [18]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# store locations
locations = main_frame[['Latitude', 'Longitude']]

# Fill NaN values and convert to float
humidity = main_frame["Humidity"].astype(float)

In [20]:
# plot gmaps
figure = gmaps.figure() # this is to make a base gmap

# add humidity layer on top of gmap
humidity_layer = gmaps


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map